In [60]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "poojasingh"

# Connect to database via CRUD Module
shelter = AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
cursor = shelter.read({})
cursor.rewind()
df = pd.DataFrame.from_records(cursor)

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
try:
    df.drop('_id', axis=1, inplace=True)
except KeyError:
    pass  # Ignore the error if '_id' column doesn't exist

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div([
        html.A(
            html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height=250, width=251)),
            href='https://www.snhu.edu' # Replace with your home page URL
        ),
        html.Center(html.B(html.H1('Pooja Singh SNHU CS-340 Dashboard'))),
    ]),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.Dropdown(
            id='rescue-type-dropdown',
            options=[
                {'label': 'All', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'}
            ],
            value='all'
        )
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here
                        selected_rows=[0],  # Select the first row by default
                        page_size=10,  # Limit the number of rows displayed per page
                        row_selectable="single",  # Enable single row selection
                        sort_action="native",  # Enable sorting
                        sort_mode="multi",  # Allow multiple columns to be sorted
                        filter_action="native",  # Enable filtering
                        style_table={'overflowX': 'scroll'},
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('rescue-type-dropdown', 'value')])
def update_dashboard(selected_rescue_type):
## FIX ME Add code to filter interactive data table with MongoDB queries

    if selected_rescue_type == 'all':
        cursor1 = shelter.read({})
    elif selected_rescue_type == 'water':
        # Filter criteria for water rescue
        # Update the regex patterns based on your data
        labRegex = re.compile(".*lab.*", re.IGNORECASE)
        chesaRegex = re.compile(".*chesa.*", re.IGNORECASE)
        newRegex = re.compile(".*newf.*", re.IGNORECASE)
        cursor1 = shelter.read({
            '$or': [
                {"breed": {'$regex': newRegex}},
                {"breed": {'$regex': chesaRegex}},
                {"breed": {'$regex': labRegex}},
            ],
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        })
    elif selected_rescue_type == 'mountain':
        # Filter criteria for mountain rescue
        # Update the regex patterns based on your data
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        alaskanRegex = re.compile(".*mala.*", re.IGNORECASE)
        oldRegex = re.compile(".*old engilish.*", re.IGNORECASE)
        huskyRegex = re.compile(".*husk.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        cursor1 = shelter.read({
            '$or': [
                {"breed": {'$regex': germanRegex}},
                {"breed": {'$regex': alaskanRegex}},
                {"breed": {'$regex': oldRegex}},
                {"breed": {'$regex': huskyRegex}},
                {"breed": {'$regex': rottRegex}},
            ],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        })
    elif selected_rescue_type == 'disaster':
        # Filter criteria for disaster rescue
        # Update the regex patterns based on your data
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        goldenRegex = re.compile(".*golden.*", re.IGNORECASE)
        bloodRegex = re.compile(".*blood.*", re.IGNORECASE)
        doberRegex = re.compile(".*dober.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        cursor1 = shelter.read({
            '$or': [
                {"breed": {'$regex': germanRegex}},
                {"breed": {'$regex': goldenRegex}},
                {"breed": {'$regex': bloodRegex}},
                {"breed": {'$regex': doberRegex}},
                {"breed": {'$regex': rottRegex}},
            ],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        })
    else:
        raise Exception("Unknown filter")
    
    cursor1.rewind()
    df_filtered = pd.DataFrame.from_records(cursor1)
    data = df_filtered.to_dict('records')
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    return data
    

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    if viewData is None or len(viewData) == 0:  # Check if viewData is None or empty
        return "No data available"
    
    dffPie = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(dffPie, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []  # Return an empty list if selected_columns is None
    else:
        return [{
            'if': { 'column_id': i },
            'background_color': '#D2F3FF'
        } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if viewData is None or len(viewData) == 0:
        return "No data available"  # Handle case where there is no data
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


{'rec_num': 2, 'age_upon_outcome': '1 year', 'animal_id': 'A725717', 'animal_type': 'Cat', 'breed': 'Domestic Shorthair Mix', 'color': 'Silver Tabby', 'date_of_birth': '2015-05-02', 'datetime': '2016-05-06 10:49:00', 'monthyear': '2016-05-06T10:49:00', 'name': '', 'outcome_subtype': 'SCRP', 'outcome_type': 'Transfer', 'sex_upon_outcome': 'Spayed Female', 'location_lat': 30.6525984560228, 'location_long': -97.7419963476444, 'age_upon_outcome_in_weeks': 52.9215277777778}
{'rec_num': 3, 'age_upon_outcome': '2 years', 'animal_id': 'A716330', 'animal_type': 'Dog', 'breed': 'Chihuahua Shorthair Mix', 'color': 'Brown/White', 'date_of_birth': '2013-11-18', 'datetime': '2015-12-28 18:43:00', 'monthyear': '2015-12-28T18:43:00', 'name': 'Frank', 'outcome_subtype': '', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Neutered Male', 'location_lat': 30.7595748121648, 'location_long': -97.5523753807133, 'age_upon_outcome_in_weeks': 110.111408730159}
{'rec_num': 1, 'age_upon_outcome': '3 years', 'ani

{'rec_num': 964, 'age_upon_outcome': '5 months', 'animal_id': 'A760903', 'animal_type': 'Cat', 'breed': 'Domestic Shorthair Mix', 'color': 'Black', 'date_of_birth': '2017-05-25', 'datetime': '2017-11-07 18:28:00', 'monthyear': '2017-11-07T18:28:00', 'name': '*Donnie', 'outcome_subtype': '', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Neutered Male', 'location_lat': 30.6455034434022, 'location_long': -97.3488085769508, 'age_upon_outcome_in_weeks': 23.8242063492063}
{'rec_num': 961, 'age_upon_outcome': '2 years', 'animal_id': 'A746658', 'animal_type': 'Dog', 'breed': 'Akita Mix', 'color': 'Red', 'date_of_birth': '2014-10-06', 'datetime': '2017-04-30 16:37:00', 'monthyear': '2017-04-30T16:37:00', 'name': 'Henry', 'outcome_subtype': '', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Neutered Male', 'location_lat': 30.3363156942325, 'location_long': -97.2815725235944, 'age_upon_outcome_in_weeks': 133.956051587302}
{'rec_num': 962, 'age_upon_outcome': '7 years', 'animal_id': 'A693120'

{'rec_num': 2281, 'age_upon_outcome': '2 years', 'animal_id': 'A694584', 'animal_type': 'Dog', 'breed': 'Pit Bull Mix', 'color': 'Brown/White', 'date_of_birth': '2012-12-31', 'datetime': '2014-12-31 12:52:00', 'monthyear': '2014-12-31T12:52:00', 'name': 'Miss Crush', 'outcome_subtype': '', 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Spayed Female', 'location_lat': 30.4325002262536, 'location_long': -97.3197699914447, 'age_upon_outcome_in_weeks': 104.362301587302}
{'rec_num': 2289, 'age_upon_outcome': '12 years', 'animal_id': 'A668298', 'animal_type': 'Cat', 'breed': 'Domestic Shorthair Mix', 'color': 'Blue/White', 'date_of_birth': '2001-12-01', 'datetime': '2013-12-06 17:25:00', 'monthyear': '2013-12-06T17:25:00', 'name': 'Cleo', 'outcome_subtype': 'Partner', 'outcome_type': 'Transfer', 'sex_upon_outcome': 'Spayed Female', 'location_lat': 30.7064113018247, 'location_long': -97.3722202001934, 'age_upon_outcome_in_weeks': 626.960813492063}
{'rec_num': 2290, 'age_upon_outcome'

{'rec_num': 4299, 'age_upon_outcome': '4 years', 'animal_id': 'A760896', 'animal_type': 'Dog', 'breed': 'Basset Hound Mix', 'color': 'Brown/White', 'date_of_birth': '2013-10-25', 'datetime': '2017-11-01 12:19:00', 'monthyear': '2017-11-01T12:19:00', 'name': '*Huck', 'outcome_subtype': 'Partner', 'outcome_type': 'Transfer', 'sex_upon_outcome': 'Neutered Male', 'location_lat': 30.6355511395561, 'location_long': -97.5215084772355, 'age_upon_outcome_in_weeks': 209.787599206349}
{'rec_num': 4297, 'age_upon_outcome': '5 years', 'animal_id': 'A662656', 'animal_type': 'Dog', 'breed': 'Doberman Pinsch/Catahoula', 'color': 'Liver/Brown Brindle', 'date_of_birth': '2008-09-06', 'datetime': '2013-10-12 18:00:00', 'monthyear': '2013-10-12T18:00:00', 'name': 'Bonnie', 'outcome_subtype': '', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Spayed Female', 'location_lat': 30.7064315482504, 'location_long': -97.5051417760428, 'age_upon_outcome_in_weeks': 266.107142857143}
{'rec_num': 4315, 'age_upon_out

{'rec_num': 5071, 'age_upon_outcome': '6 years', 'animal_id': 'A672198', 'animal_type': 'Dog', 'breed': 'Pit Bull Mix', 'color': 'Red/White', 'date_of_birth': '2008-02-05', 'datetime': '2014-02-11 16:29:00', 'monthyear': '2014-02-11T16:29:00', 'name': '*Teddy', 'outcome_subtype': 'Suffering', 'outcome_type': 'Euthanasia', 'sex_upon_outcome': 'Intact Male', 'location_lat': 30.693244436298, 'location_long': -97.68872615796, 'age_upon_outcome_in_weeks': 314.098115079365}
{'rec_num': 5083, 'age_upon_outcome': '14 years', 'animal_id': 'A765807', 'animal_type': 'Dog', 'breed': 'Shih Tzu Mix', 'color': 'White/Tan', 'date_of_birth': '2004-01-27', 'datetime': '2018-01-29 12:18:00', 'monthyear': '2018-01-29T12:18:00', 'name': '', 'outcome_subtype': 'Partner', 'outcome_type': 'Transfer', 'sex_upon_outcome': 'Neutered Male', 'location_lat': 30.4327325467381, 'location_long': -97.6687512153627, 'age_upon_outcome_in_weeks': 730.930357142857}
{'rec_num': 5089, 'age_upon_outcome': '3 years', 'animal_i

...

{'rec_num': 4297, 'age_upon_outcome': '5 years', 'animal_id': 'A662656', 'animal_type': 'Dog', 'breed': 'Doberman Pinsch/Catahoula', 'color': 'Liver/Brown Brindle', 'date_of_birth': '2008-09-06', 'datetime': '2013-10-12 18:00:00', 'monthyear': '2013-10-12T18:00:00', 'name': 'Bonnie', 'outcome_subtype': '', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Spayed Female', 'location_lat': 30.7064315482504, 'location_long': -97.5051417760428, 'age_upon_outcome_in_weeks': 266.107142857143}
{'rec_num': 4315, 'age_upon_outcome': '1 year', 'animal_id': 'A763722', 'animal_type': 'Cat', 'breed': 'Domestic Shorthair Mix', 'color': 'Brown Tabby', 'date_of_birth': '2016-12-14', 'datetime': '2017-12-15 13:01:00', 'monthyear': '2017-12-15T13:01:00', 'name': '', 'outcome_subtype': 'Partner', 'outcome_type': 'Transfer', 'sex_upon_outcome': 'Intact Male', 'location_lat': 30.4040980719363, 'location_long': -97.6607964624153, 'age_upon_outcome_in_weeks': 52.3631944444444}
{'rec_num': 4301, 'age_upon_outco

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'rec_num': 3905, 'age_upon_outcome': '1 year', 'animal_id': 'A750806', 'animal_type': 'Dog', 'breed': 'Chinese Sharpei/Labrador Retriever', 'color': 'Chocolate', 'date_of_birth': '2016-05-31', 'datetime': '2017-09-26 14:46:00', 'monthyear': '2017-09-26T14:46:00', 'name': 'Precious', 'outcome_subtype': '', 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Intact Female', 'location_lat': 30.6229710455683, 'location_long': -97.3445078033179, 'age_upon_outcome_in_weeks': 69.0878968253968}
{'rec_num': 6128, 'age_upon_outcome': '9 months', 'animal_id': 'A745478', 'animal_type': 'Dog', 'breed': 'German Shepherd/Labrador Retriever', 'color': 'Tricolor', 'date_of_birth': '2016-06-18', 'datetime': '2017-03-24 12:03:00', 'monthyear': '2017-03-24T12:03:00', 'name': '*Myra', 'outcome_subtype': 'Partner', 'outcome_type': 'Transfer', 'sex_upon_outcome': 'Intact Female', 'location_lat': 30.7007209771159, 'location_long': -97.247241975955, 'age_upon_outcome_in_weeks': 39.928869047619}
{'rec_num'

{'rec_num': 2077, 'age_upon_outcome': '1 year', 'animal_id': 'A745826', 'animal_type': 'Dog', 'breed': 'Alaskan Husky Mix', 'color': 'White/White', 'date_of_birth': '2016-03-24', 'datetime': '2017-03-27 16:28:00', 'monthyear': '2017-03-27T16:28:00', 'name': 'Zeke', 'outcome_subtype': '', 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Intact Male', 'location_lat': 30.5468458839857, 'location_long': -97.4375169248344, 'age_upon_outcome_in_weeks': 52.6694444444444}
{'rec_num': 9010, 'age_upon_outcome': '1 year', 'animal_id': 'A725008', 'animal_type': 'Dog', 'breed': 'Alaskan Husky Mix', 'color': 'Black/White', 'date_of_birth': '2015-04-24', 'datetime': '2017-02-08 17:44:00', 'monthyear': '2017-02-08T17:44:00', 'name': 'Bloo', 'outcome_subtype': '', 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Intact Male', 'location_lat': 30.2914605679945, 'location_long': -97.380326878943, 'age_upon_outcome_in_weeks': 93.8198412698413}
{'rec_num': 5315, 'age_upon_outcome': '2 years', 

{'rec_num': 5411, 'age_upon_outcome': '5 months', 'animal_id': 'A669643', 'animal_type': 'Dog', 'breed': 'Australian Cattle Dog/Bloodhound', 'color': 'Brown Brindle/White', 'date_of_birth': '2013-07-24', 'datetime': '2014-01-05 13:10:00', 'monthyear': '2014-01-05T13:10:00', 'name': 'Dale', 'outcome_subtype': '', 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Intact Male', 'location_lat': 30.2852781393318, 'location_long': -97.3924381749523, 'age_upon_outcome_in_weeks': 23.6498015873016}
{'rec_num': 3767, 'age_upon_outcome': '4 years', 'animal_id': 'A712291', 'animal_type': 'Dog', 'breed': 'Bloodhound', 'color': 'Red', 'date_of_birth': '2011-09-20', 'datetime': '2015-09-22 15:43:00', 'monthyear': '2015-09-22T15:43:00', 'name': 'Boomer', 'outcome_subtype': '', 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Intact Male', 'location_lat': 30.2709983761287, 'location_long': -97.5923916912722, 'age_upon_outcome_in_weeks': 209.093551587302}
{'rec_num': 1918, 'age_upon_outcome